코드 출처: https://github.com/deepseasw/seq2seq_chatbot<br>

In [10]:
from tensorflow import keras
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers, losses, metrics
from tensorflow.keras import preprocessing

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import os
import re

from konlpy.tag import Okt

<br>
<br>

# 데이터 로드

Seq2Seq에서의 임베딩이 이전 예제와 다른 점은 아래와 같이 태그를 사용한다는 것입니다.<br>
임베딩의 0~3번째에 각각 PADDING, START, END, OOV 태그를 넣습니다.<br>
사실 그냥 똑같은 단어라고 보시면 됩니다. 다만 이 단어들이 Seq2Seq의 동작을 제어합니다. <br>
<br>
예를 들어, 디코더 입력에 START가 들어가면 디코딩의 시작을 의미합니다. 반대로 디코더 출력에 END가 나오면 디코딩을 종료합니다.<br>

In [11]:
# 태그 단어
PAD = "<PADDING>"   # 패딩
STA = "<START>"     # 시작
END = "<END>"       # 끝
OOV = "<OOV>"       # 없는 단어(Out of Vocabulary)

# 태그 인덱스
PAD_INDEX = 0
STA_INDEX = 1
END_INDEX = 2
OOV_INDEX = 3

# 데이터 타입
ENCODER_INPUT  = 0
DECODER_INPUT  = 1
DECODER_TARGET = 2

# 한 문장에서 단어 시퀀스의 최대 개수
max_sequences = 30

# 임베딩 벡터 차원
embedding_dim = 100

# LSTM 히든레이어 차원
lstm_hidden_dim = 128

# 정규 표현식 필터
RE_FILTER = re.compile("[.,!?\"':;~()]")

# 챗봇 데이터 로드
chatbot_data = pd.read_csv('ChatbotData.csv', encoding='utf-8')
question, answer = list(chatbot_data['Q']), list(chatbot_data['A'])

챗봇의 훈련을 위해서 송영숙님이 공개한 한글 데이터셋을 로드합니다.<br>
질문과 대답, 감정 등 총 3개의 항목으로 구성되어 있습니다.<br>
감정 분류는 Seq2Seq에 필요가 없기 때문에 사용하지 않습니다.<br>
https://github.com/songys/Chatbot_data<br>


In [12]:
# 데이터 개수
len(question)

11824

In [13]:
# 데이터의 일부만 학습에 사용
question = question[:100]
answer = answer[:100]

# 챗봇 데이터 출력
for i in range(10):
    print('Q : ' + question[i])
    print('A : ' + answer[i])
    print()


Q : 12시 땡!
A : 하루가 또 가네요.

Q : 1지망 학교 떨어졌어
A : 위로해 드립니다.

Q : 3박4일 놀러가고 싶다
A : 여행은 언제나 좋죠.

Q : 3박4일 정도 놀러가고 싶다
A : 여행은 언제나 좋죠.

Q : PPL 심하네
A : 눈살이 찌푸려지죠.

Q : SD카드 망가졌어
A : 다시 새로 사는 게 마음 편해요.

Q : SD카드 안돼
A : 다시 새로 사는 게 마음 편해요.

Q : SNS 맞팔 왜 안하지ㅠㅠ
A : 잘 모르고 있을 수도 있어요.

Q : SNS 시간낭비인 거 아는데 매일 하는 중
A : 시간을 정하고 해보세요.

Q : SNS 시간낭비인데 자꾸 보게됨
A : 시간을 정하고 해보세요.



<br>
<br>

# 단어 사전 생성

In [14]:
# 형태소분석 함수
def pos_tag(sentences):
    
    # KoNLPy 형태소분석기 설정
    tagger = Okt()
    
    # 문장 품사 변수 초기화
    sentences_pos = []
    
    # 모든 문장 반복
    for sentence in sentences:
        # 특수기호 제거
        # RE_FILTER에 해당되는 정규표현식 char에 대하여 "" ()로 바꾸어라
        sentence = re.sub(RE_FILTER, "", sentence)
        
        # 배열인 형태소분석의 출력을 띄어쓰기로 구분하여 붙임
        sentence = " ".join(tagger.morphs(sentence))
        sentences_pos.append(sentence)
        
    return sentences_pos

In [15]:
# 형태소분석 수행
question = pos_tag(question)
answer = pos_tag(answer)

# 형태소분석으로 변환된 챗봇 데이터 출력
for i in range(10):
    print('Q : ' + question[i])
    print('A : ' + answer[i])
    print()

Q : 12시 땡
A : 하루 가 또 가네요

Q : 1 지망 학교 떨어졌어
A : 위로 해 드립니다

Q : 3 박 4일 놀러 가고 싶다
A : 여행 은 언제나 좋죠

Q : 3 박 4일 정도 놀러 가고 싶다
A : 여행 은 언제나 좋죠

Q : PPL 심하네
A : 눈살 이 찌푸려지죠

Q : SD 카드 망가졌어
A : 다시 새로 사는 게 마음 편해요

Q : SD 카드 안 돼
A : 다시 새로 사는 게 마음 편해요

Q : SNS 맞팔 왜 안 하지 ㅠㅠ
A : 잘 모르고 있을 수도 있어요

Q : SNS 시간 낭비 인 거 아는데 매일 하는 중
A : 시간 을 정 하고 해보세요

Q : SNS 시간 낭비 인데 자꾸 보게 됨
A : 시간 을 정 하고 해보세요



In [16]:
# 질문과 대답 문장들을 하나로 합침
sentences = []
sentences.extend(question)
sentences.extend(answer)

words = []

# 단어들의 배열 생성
for sentence in sentences:
    for word in sentence.split():
        words.append(word)

# 길이가 0인 단어는 삭제
words = [word for word in words if len(word) > 0]

# 중복된 단어 삭제
words = list(set(words))

# 제일 앞에 태그 단어 삽입
words[:0] = [PAD, STA, END, OOV]

질문과 대답 문장들을 합쳐서 전체 단어 사전을 만듭니다.<br>
자연어처리에서는 항상 이렇게 단어를 인덱스에 따라 정리를 해야 합니다.<br>
<br>
그래야지 문장을 인덱스 배열로 바꿔서 임베딩 레이어에 넣을 수 있습니다.<br>
또한 모델의 출력에서 나온 인덱스를 다시 단어로 변환하는데도 필요합니다.<br>
<br>

In [17]:
# 단어 개수
len(words)

454

In [18]:
# 단어 출력
words

['<PADDING>',
 '<START>',
 '<END>',
 '<OOV>',
 '있어도',
 '정',
 '은',
 '가지',
 '식혀주세요',
 '다음',
 '단',
 '낮잠',
 '놓고',
 '알',
 '하루',
 '하',
 '위로',
 '정말',
 '함께',
 'PPL',
 '갈',
 '이에요',
 '이다',
 '없죠',
 '부터',
 '관리',
 '처럼',
 '시켜야지',
 '약',
 '먹었는데',
 '싫어요',
 '일',
 '있어',
 '간장',
 '요',
 '가난한',
 '4일',
 '사랑',
 '했잖아',
 '개인',
 '곧',
 '입어볼까',
 '간다',
 '때',
 '새로',
 '습관',
 '여행',
 '집',
 '끄고',
 '물어보세요',
 '인거',
 '싫어하지',
 '사이',
 '엉망',
 '그런거니',
 '걔',
 '수도',
 '비싼데',
 '싫어',
 '옴',
 '나왔다',
 '가야',
 '맞팔',
 '막',
 '비',
 '되나',
 '당신',
 '켜놓고',
 '끼리',
 '확실한',
 '눈살',
 '살까',
 '간식',
 '그게',
 '했길',
 '잊고',
 '나쁜',
 '하는',
 '집어서',
 '예쁘게',
 '3',
 '결정',
 '지망',
 '뭐',
 '갑자기',
 '사는게',
 '연인',
 '해볼까',
 '가장',
 '자꾸',
 '누구',
 '어디',
 '의',
 '같은',
 '이야',
 '이나',
 '서먹해졌어',
 '하지',
 '개학',
 '떨리는',
 '말랭이',
 '까',
 '야',
 '좋아',
 '세수',
 '개념',
 '사는',
 '중',
 '하면',
 '살쪄도',
 '먹고',
 '마음',
 '키우고',
 '있을',
 '첫인상',
 '먹어야지',
 '닮아서',
 '따뜻하게',
 '간접흡연',
 '설움',
 '추천',
 '패턴',
 '혼자',
 '즐거운',
 '찌푸려지죠',
 '이야기',
 '취미',
 '땡',
 '가네요',
 '이럴',
 '이랑',
 '돈',
 '낭비',
 'ㅠㅠ',
 '키울까',
 '살찐',
 

In [19]:
# 단어와 인덱스의 딕셔너리 생성, Series.count_values()도 가능
word_to_index = {word: index for index, word in enumerate(words)}
index_to_word = {index: word for index, word in enumerate(words)}

In [20]:
# 단어 -> 인덱스
# 문장을 인덱스로 변환하여 모델 입력으로 사용
word_to_index

{'<PADDING>': 0,
 '<START>': 1,
 '<END>': 2,
 '<OOV>': 3,
 '있어도': 4,
 '정': 5,
 '은': 6,
 '가지': 7,
 '식혀주세요': 8,
 '다음': 9,
 '단': 10,
 '낮잠': 11,
 '놓고': 12,
 '알': 13,
 '하루': 14,
 '하': 15,
 '위로': 16,
 '정말': 17,
 '함께': 18,
 'PPL': 19,
 '갈': 20,
 '이에요': 21,
 '이다': 22,
 '없죠': 23,
 '부터': 24,
 '관리': 25,
 '처럼': 26,
 '시켜야지': 27,
 '약': 28,
 '먹었는데': 29,
 '싫어요': 30,
 '일': 31,
 '있어': 32,
 '간장': 33,
 '요': 34,
 '가난한': 35,
 '4일': 36,
 '사랑': 37,
 '했잖아': 38,
 '개인': 39,
 '곧': 40,
 '입어볼까': 41,
 '간다': 42,
 '때': 43,
 '새로': 44,
 '습관': 45,
 '여행': 46,
 '집': 47,
 '끄고': 48,
 '물어보세요': 49,
 '인거': 50,
 '싫어하지': 51,
 '사이': 52,
 '엉망': 53,
 '그런거니': 54,
 '걔': 55,
 '수도': 56,
 '비싼데': 57,
 '싫어': 58,
 '옴': 59,
 '나왔다': 60,
 '가야': 61,
 '맞팔': 62,
 '막': 63,
 '비': 64,
 '되나': 65,
 '당신': 66,
 '켜놓고': 67,
 '끼리': 68,
 '확실한': 69,
 '눈살': 70,
 '살까': 71,
 '간식': 72,
 '그게': 73,
 '했길': 74,
 '잊고': 75,
 '나쁜': 76,
 '하는': 77,
 '집어서': 78,
 '예쁘게': 79,
 '3': 80,
 '결정': 81,
 '지망': 82,
 '뭐': 83,
 '갑자기': 84,
 '사는게': 85,
 '연인': 86,
 '해볼까': 87,
 '가장': 88,


In [21]:
# 인덱스 -> 단어
# 모델의 예측 결과인 인덱스를 문장으로 변환시 사용
index_to_word


{0: '<PADDING>',
 1: '<START>',
 2: '<END>',
 3: '<OOV>',
 4: '있어도',
 5: '정',
 6: '은',
 7: '가지',
 8: '식혀주세요',
 9: '다음',
 10: '단',
 11: '낮잠',
 12: '놓고',
 13: '알',
 14: '하루',
 15: '하',
 16: '위로',
 17: '정말',
 18: '함께',
 19: 'PPL',
 20: '갈',
 21: '이에요',
 22: '이다',
 23: '없죠',
 24: '부터',
 25: '관리',
 26: '처럼',
 27: '시켜야지',
 28: '약',
 29: '먹었는데',
 30: '싫어요',
 31: '일',
 32: '있어',
 33: '간장',
 34: '요',
 35: '가난한',
 36: '4일',
 37: '사랑',
 38: '했잖아',
 39: '개인',
 40: '곧',
 41: '입어볼까',
 42: '간다',
 43: '때',
 44: '새로',
 45: '습관',
 46: '여행',
 47: '집',
 48: '끄고',
 49: '물어보세요',
 50: '인거',
 51: '싫어하지',
 52: '사이',
 53: '엉망',
 54: '그런거니',
 55: '걔',
 56: '수도',
 57: '비싼데',
 58: '싫어',
 59: '옴',
 60: '나왔다',
 61: '가야',
 62: '맞팔',
 63: '막',
 64: '비',
 65: '되나',
 66: '당신',
 67: '켜놓고',
 68: '끼리',
 69: '확실한',
 70: '눈살',
 71: '살까',
 72: '간식',
 73: '그게',
 74: '했길',
 75: '잊고',
 76: '나쁜',
 77: '하는',
 78: '집어서',
 79: '예쁘게',
 80: '3',
 81: '결정',
 82: '지망',
 83: '뭐',
 84: '갑자기',
 85: '사는게',
 86: '연인',
 87: '해볼까',
 88: '가장',


<br>
<br>

# 전처리

In [22]:
# 문장을 인덱스로 변환
def convert_text_to_index(sentences, vocabulary, type): 
    
    sentences_index = []
    
    # 모든 문장에 대해서 반복
    for sentence in sentences:
        sentence_index = []
        
        # 디코더 입력일 경우 맨 앞에 START 태그 추가
        if type == DECODER_INPUT:
            sentence_index.extend([vocabulary[STA]])
        
        # 문장의 단어들을 띄어쓰기로 분리
        for word in sentence.split():
            if vocabulary.get(word) is not None:
                # 사전에 있는 단어면 해당 인덱스를 추가
                sentence_index.extend([vocabulary[word]])
            else:
                # 사전에 없는 단어면 OOV 인덱스를 추가
                sentence_index.extend([vocabulary[OOV]])

        # 최대 길이 검사
        if type == DECODER_TARGET:
            # 디코더 목표일 경우 맨 뒤에 END 태그 추가
            if len(sentence_index) >= max_sequences:
                sentence_index = sentence_index[:max_sequences-1] + [vocabulary[END]]
            else:
                sentence_index += [vocabulary[END]]
        else:
            if len(sentence_index) > max_sequences:
                sentence_index = sentence_index[:max_sequences]
            
        # 최대 길이에 없는 공간은 패딩 인덱스로 채움
        sentence_index += (max_sequences - len(sentence_index)) * [vocabulary[PAD]]
        
        # 문장의 인덱스 배열을 추가
        sentences_index.append(sentence_index)

    return np.asarray(sentences_index)

Seq2Seq에서는 학습시 다음과 같이 총 3개의 데이터가 필요합니다.

인코더 입력 : 12시 땡<br>
디코더 입력 : START 하루 가 또 가네요<br>
디코더 출력 : 하루 가 또 가네요 END
<br>
원래 Seq2Seq는 디코더의 현재 출력이 디코더의 다음 입력으로 들어갑니다.<br>
다만 학습에서는 굳이 이렇게 하지 않고 디코더 입력과 디코더 출력의 데이터를 각각 만듭니다. <br>
<br>
그러나 예측시에는 이런 방식이 불가능합니다.<br>
출력값을 미리 알지 못하기 때문에, 디코더 입력을 사전에 생성할 수가 없습니다.<br>
이런 문제를 해결하기 위해 훈련 모델과 예측 모델을 따로 구성해야 합니다.<br>
모델 생성 부분에서 다시 자세히 설명을 드리겠습니다.<br>
<br>
<br>

In [23]:
# 인코더 입력 인덱스 변환
x_encoder = convert_text_to_index(question, word_to_index, ENCODER_INPUT)

print('첫번째 question:', question[0])
# 첫 번째 인코더 입력 출력 (12시 땡)
print('첫번째 Encoder 입력:',x_encoder[0])
print()

# 출력된 125, 308은 word의 index
print('index_to_word[ 125 ]:', index_to_word[ 125 ])
print('index_to_word[ 308 ]:', index_to_word[ 308 ])

첫번째 question: 12시 땡
첫번째 Encoder 입력: [417 127   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0]

index_to_word[ 125 ]: 이야기
index_to_word[ 308 ]: 물어봐서


In [24]:
# 디코더 입력 인덱스 변환
x_decoder = convert_text_to_index(answer, word_to_index, DECODER_INPUT)

print('첫번째 answer:', answer[0])
print('첫번째 Decoder input:', x_decoder[0])
print()

# 출력된 1, 259, 223, 114, 74는 word의 index
print('index_to_word[   1 ]:', index_to_word[   1 ])
print('index_to_word[ 259 ]:', index_to_word[ 259 ])
print('index_to_word[ 223 ]:', index_to_word[ 223 ])
print('index_to_word[ 114 ]:', index_to_word[ 114 ])
print('index_to_word[  74 ]:', index_to_word[  74 ])

첫번째 answer: 하루 가 또 가네요
첫번째 Decoder input: [  1  14 436 151 128   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0]

index_to_word[   1 ]: <START>
index_to_word[ 259 ]: 책임질
index_to_word[ 223 ]: 해보여
index_to_word[ 114 ]: 첫인상
index_to_word[  74 ]: 했길


In [25]:
# 디코더 목표 인덱스 변환
y_decoder = convert_text_to_index(answer, word_to_index, DECODER_TARGET)

# 첫 번째 디코더 목표 출력 (하루 가 또 가네요 END)
y_decoder[0]

print('첫번째 answer:', answer[0])
print('첫번쨰 Decoder onput:', y_decoder[0])
print()

# 출력된 1, 259, 223, 114, 74는 word의 index
print('index_to_word[ 259 ]:', index_to_word[ 259 ])
print('index_to_word[ 223 ]:', index_to_word[ 223 ])
print('index_to_word[ 114 ]:', index_to_word[ 114 ])
print('index_to_word[  74 ]:', index_to_word[  74 ])
print('index_to_word[   2 ]:', index_to_word[   2 ])

첫번째 answer: 하루 가 또 가네요
첫번쨰 Decoder onput: [ 14 436 151 128   2   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0]

index_to_word[ 259 ]: 책임질
index_to_word[ 223 ]: 해보여
index_to_word[ 114 ]: 첫인상
index_to_word[  74 ]: 했길
index_to_word[   2 ]: <END>


In [26]:
# 원핫인코딩 초기화
one_hot_data = np.zeros((len(y_decoder), max_sequences, len(words)))
one_hot_data[0]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [27]:
#one_hot_data의 첫 row에 259번째 column에 '하루'
#one_hot_data의 첫 row에 223번째 column에 '가'
#one_hot_data의 첫 row에 114번째 column에 '또'.... 를 1로 변경
y_decoder[0]

array([ 14, 436, 151, 128,   2,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0])

In [28]:
# 디코더 목표를 원핫인코딩으로 변환
# 학습시 입력은 인덱스이지만, 출력은 원핫인코딩 형식임
for i, sequence in enumerate(y_decoder):
    for j, index in enumerate(sequence):
        one_hot_data[i, j, index] = 1

# 디코더 목표 설정
y_decoder = one_hot_data

# 첫 번째 디코더 목표 출력
y_decoder[0]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]])

In [29]:
# 259번째 index (하루) 가 1임
y_decoder[0][0][112]

0.0

인코더 입력과 디코더 입력은 임베딩 레이어에 들어가는 인덱스 배열입니다.<br>
반면에 디코더 출력은 원핫인코딩 형식이 되어야 합니다.<br>
디코더의 마지막 Dense 레이어에서 softmax로 나오기 때문입니다.<br>

<br>
<br>

# 모델 생성

In [30]:
#--------------------------------------------
# 훈련 모델 인코더 정의
#--------------------------------------------

# 입력 문장의 인덱스 시퀀스를 입력으로 받음
encoder_inputs = layers.Input(shape=(None,))

# 임베딩 레이어
encoder_outputs = layers.Embedding(len(words), embedding_dim)(encoder_inputs)

# return_state가 True면 상태값 리턴
# LSTM은 state_h(hidden state)와 state_c(cell state) 2개의 상태 존재
encoder_outputs, state_h, state_c = layers.LSTM(lstm_hidden_dim,
                                                dropout=0.1,
                                                recurrent_dropout=0.5,
                                                return_state=True)(encoder_outputs)

# 히든 상태와 셀 상태를 하나로 묶음
# Decoder의 initial state에 넣어주기 위함
# 즉, input sentence의 모든 정보를 통해 Decoding 하기 위함
encoder_states = [state_h, state_c]


In [31]:
encoder_outputs

<KerasTensor: shape=(None, 128) dtype=float32 (created by layer 'lstm')>

In [32]:
encoder_states

[<KerasTensor: shape=(None, 128) dtype=float32 (created by layer 'lstm')>,
 <KerasTensor: shape=(None, 128) dtype=float32 (created by layer 'lstm')>]

In [33]:
#--------------------------------------------
# 훈련 모델 디코더 정의
#--------------------------------------------

# 목표 문장의 인덱스 시퀀스를 입력으로 받음
decoder_inputs = layers.Input(shape=(None,))

# 임베딩 레이어
decoder_embedding = layers.Embedding(len(words), embedding_dim)
decoder_outputs = decoder_embedding(decoder_inputs)

# 인코더와 달리 return_sequences를 True로 설정하여 모든 타임 스텝 출력값 리턴
# 모든 타임 스텝의 출력값들을 다음 레이어의 Dense()로 처리하기 위함
decoder_lstm = layers.LSTM(lstm_hidden_dim,
                           dropout=0.1,
                           recurrent_dropout=0.5,
                           return_state=True,
                           return_sequences=True)

# initial_state를 인코더의 상태로 초기화
decoder_outputs, _, _ = decoder_lstm(decoder_outputs,
                                     initial_state=encoder_states)

# 단어의 개수만큼 노드의 개수를 설정하여 원핫 형식으로 각 단어 인덱스를 출력
decoder_dense = layers.Dense(len(words), activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)


지금까지의 예제는 Sequential 방식의 모델이었습니다.<br>
하지만 이번에는 함수형 API 모델을 사용했습니다.<br>
인코더와 디코더가 따로 분리되어야 하는데, 단순히 레이어를 추가하여 붙이는 순차형으로는 구현이 불가능하기 때문입니다. <br>
<br>


In [34]:
#--------------------------------------------
# 훈련 모델 정의
#--------------------------------------------

# 입력과 출력으로 함수형 API 모델 생성
model = models.Model([encoder_inputs, decoder_inputs], decoder_outputs)

# 학습 방법 설정
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['acc'])    

Model() 함수로 입력과 출력을 따로 설정하여 모델을 만듭니다.<br>
그다음 compile과 fit은 이전과 동일하게 적용하시면 됩니다.<br>
<br>
<br>

In [35]:
#--------------------------------------------
#  예측 모델 인코더 정의
#--------------------------------------------

# 훈련 모델의 인코더 상태를 사용하여 예측 모델 인코더 설정
encoder_model = models.Model(encoder_inputs, encoder_states)

#--------------------------------------------
# 예측 모델 디코더 정의
#--------------------------------------------

# 예측시에는 훈련시와 달리 타임 스텝을 한 단계씩 수행
# 매번 이전 디코더 상태를 입력으로 받아서 새로 설정
decoder_state_input_h = layers.Input(shape=(lstm_hidden_dim,))
decoder_state_input_c = layers.Input(shape=(lstm_hidden_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]    

# 임베딩 레이어
decoder_outputs = decoder_embedding(decoder_inputs)

# LSTM 레이어
decoder_outputs, state_h, state_c = decoder_lstm(decoder_outputs,
                                                 initial_state=decoder_states_inputs)

# 히든 상태와 셀 상태를 하나로 묶음
decoder_states = [state_h, state_c]

# Dense 레이어를 통해 원핫 형식으로 각 단어 인덱스를 출력
decoder_outputs = decoder_dense(decoder_outputs)

# 예측 모델 디코더 설정
decoder_model = models.Model([decoder_inputs] + decoder_states_inputs,
                      [decoder_outputs] + decoder_states)

예측 모델은 이미 학습된 훈련 모델의 레이어들을 그대로 재사용합니다. 예측 모델 인코더는 훈련 모델 인코더과 동일합니다. 그러나 예측 모델 디코더는 매번 LSTM 상태값을 입력으로 받습니다. 또한 디코더의 LSTM 상태를 출력값과 같이 내보내서, 다음 번 입력에 넣습니다. 

이렇게 하는 이유는 LSTM을 딱 한번의 타임 스텝만 실행하기 때문입니다. 그래서 매번 상태값을 새로 초기화 해야 합니다. 이와 반대로 훈련할때는 문장 전체를 계속 LSTM으로 돌리기 때문에 자동으로 상태값이 전달됩니다. 

In [36]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 embedding (Embedding)          (None, None, 100)    45400       ['input_1[0][0]']                
                                                                                                  
 embedding_1 (Embedding)        (None, None, 100)    45400       ['input_2[0][0]']                
                                                                                              

In [37]:
encoder_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding (Embedding)       (None, None, 100)         45400     
                                                                 
 lstm (LSTM)                 [(None, 128),             117248    
                              (None, 128),                       
                              (None, 128)]                       
                                                                 
Total params: 162,648
Trainable params: 162,648
Non-trainable params: 0
_________________________________________________________________


In [38]:
# Decoder_model에는 
decoder_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 embedding_1 (Embedding)        (None, None, 100)    45400       ['input_2[0][0]']                
                                                                                                  
 input_3 (InputLayer)           [(None, 128)]        0           []                               
                                                                                                  
 input_4 (InputLayer)           [(None, 128)]        0           []                               
                                                                                            

<br>
<br>

# 훈련 및 테스트

In [39]:
# 인덱스를 문장으로 변환
def convert_index_to_text(indexs, vocabulary): 
    
    sentence = ''
    
    # 모든 문장에 대해서 반복
    for index in indexs:
        if index == END_INDEX:
            # 종료 인덱스면 중지
            break;
        elif vocabulary.get(index) is not None:
            # 사전에 있는 인덱스면 해당 단어를 추가
            sentence += vocabulary[index]
        else:
            # 사전에 없는 인덱스면 OOV 단어를 추가
            sentence += vocabulary[OOV_INDEX]
            
        # 빈칸 추가
        sentence += ' '

    return sentence

In [40]:
# 에폭 반복
for epoch in range(20):
    print('Total Epoch :', epoch + 1)

    # 훈련 시작
    history = model.fit([x_encoder, x_decoder],
                        y_decoder,
                        epochs=100,
                        batch_size=64,
                        verbose=0)
    
    # 정확도와 손실 출력
    print('accuracy :', history.history['acc'][-1])
    print('loss :', history.history['loss'][-1])
    
    # 문장 예측 테스트
    # (3 박 4일 놀러 가고 싶다) -> (여행 은 언제나 좋죠)
    input_encoder = x_encoder[2].reshape(1, x_encoder[2].shape[0])
    input_decoder = x_decoder[2].reshape(1, x_decoder[2].shape[0])
    results = model.predict([input_encoder, input_decoder])
    
    # 결과의 원핫인코딩 형식을 인덱스로 변환
    # 1축을 기준으로 가장 높은 값의 위치를 구함
    indexs = np.argmax(results[0], 1) 
    
    # 인덱스를 문장으로 변환
    sentence = convert_index_to_text(indexs, index_to_word)
    print(sentence)
    print()


Total Epoch : 1


KeyboardInterrupt: 

학습이 진행될수록 예측 문장이 제대로 생성되는 것을 볼 수 있습니다. 다만 여기서의 예측은 단순히 테스트를 위한 것이라, 인코더 입력과 디코더 입력 데이터가 동시에 사용됩니다. 아래 문장 생성에서는 예측 모델을 적용하기 때문에, 오직 인코더 입력 데이터만 집어 넣습니다.
<br>
<br>
<br>

In [41]:
# 모델 저장
encoder_model.save('./model/seq2seq_chatbot_encoder_model.h5')
decoder_model.save('./model/seq2seq_chatbot_decoder_model.h5')

# 인덱스 저장
with open('./model/word_to_index.pkl', 'wb') as f:
    pickle.dump(word_to_index, f, pickle.HIGHEST_PROTOCOL)
with open('./model/index_to_word.pkl', 'wb') as f:
    pickle.dump(index_to_word, f, pickle.HIGHEST_PROTOCOL)    

<br>
<br>

# 문장 생성

In [42]:
# 모델 파일 로드
encoder_model = models.load_model('./model/seq2seq_chatbot_encoder_model.h5')
decoder_model = models.load_model('./model/seq2seq_chatbot_decoder_model.h5')

# 인덱스 파일 로드
with open('./model/word_to_index.pkl', 'rb') as f:
    word_to_index = pickle.load(f)
with open('./model/index_to_word.pkl', 'rb') as f:
    index_to_word = pickle.load(f)    

In [43]:
# 예측을 위한 입력 생성
def make_predict_input(sentence):

    sentences = []
    sentences.append(sentence)
    sentences = pos_tag(sentences)
    input_seq = convert_text_to_index(sentences, word_to_index, ENCODER_INPUT)
    
    return input_seq

In [44]:
# 텍스트 생성
def generate_text(input_seq):
    
    # 입력을 인코더에 넣어 마지막 상태 구함
    states = encoder_model.predict(input_seq)

    # 목표 시퀀스 초기화
    target_seq = np.zeros((1, 1))
    
    # 목표 시퀀스의 첫 번째에 <START> 태그 추가
    target_seq[0, 0] = STA_INDEX
    
    # 인덱스 초기화
    indexs = []
    
    # 디코더 타임 스텝 반복
    while 1:
        # 디코더로 현재 타임 스텝 출력 구함
        # 처음에는 인코더 상태를, 다음부터 이전 디코더 상태로 초기화
        decoder_outputs, state_h, state_c = decoder_model.predict(
                                                [target_seq] + states)

        # 결과의 원핫인코딩 형식을 인덱스로 변환
        index = np.argmax(decoder_outputs[0, 0, :])
        indexs.append(index)
        
        # 종료 검사
        if index == END_INDEX or len(indexs) >= max_sequences:
            break

        # 목표 시퀀스를 바로 이전의 출력으로 설정
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = index
        
        # 디코더의 이전 상태를 다음 디코더 예측에 사용
        states = [state_h, state_c]

    # 인덱스를 문장으로 변환
    sentence = convert_index_to_text(indexs, index_to_word)
        
    return sentence

제일 첫 단어는 START로 시작합니다. 그리고 출력으로 나온 인덱스를 디코더 입력으로 넣고 다시 예측을 반복합니다. 상태값을 받아 다시 입력으로 같이 넣는 것에 주의하시기 바랍니다. END 태그가 나오면 문장 생성을 종료합니다.
<br>
<br>
<br>

In [45]:
# 문장을 인덱스로 변환
input_seq = make_predict_input('휴강이 좋아요')
input_seq


array([[  3, 423, 445,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0]])

In [46]:
# 예측 모델로 텍스트 생성
sentence = generate_text(input_seq)
sentence


'저 도 '

데이터셋에 있는 문장과 똑같은 입력을 넣으니, 역시 정확히 일치하는 답변이 출력되었습니다.
<br>
<br>
<br>

In [69]:
# 문장을 인덱스로 변환
input_seq = make_predict_input('')
input_seq


array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0]])

In [70]:
# 예측 모델로 텍스트 생성
sentence = generate_text(input_seq)
sentence


'저 도 '

최고의 강의입니다
<br>
<br>
<br>

In [59]:
# 문장을 인덱스로 변환
input_seq = make_predict_input('4박5일 욜로가려고요')
input_seq

array([[  3, 204,   3,   3, 438,   3,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0]])

In [60]:
# 예측 모델로 텍스트 생성
sentence = generate_text(input_seq)
sentence

'저 도 '

하지만 데이터셋에 없던 '4박5일, 욜로'로 입력을 수정하니, 전혀 다른 문장이 출력되었습니다.<br>
이는 우리가 데이터의 일부인 100개 문장만 학습했기 때문입니다.<br>
데이터의 개수를 늘려서 훈련할수록 일반화 능력이 더욱 높아집니다.